In [1]:
import os
from glob import glob
import cv2
import json
from tqdm import tqdm

In [2]:
def createDirectory(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print("Error: Failed to create the directory.")

In [3]:
def label_make(i, labeling):
    label = '99'
    if i == '아우터':
        if '카테고리' in labeling:
            if labeling['카테고리'] == '베스트':
                label = '3'
            elif labeling['카테고리'] == '코트':
                label = '7'
            else:
                label = '2'
        # else:
        #     label = '2'
    elif i == '상의':
        if '카테고리' in labeling:
            if labeling['카테고리'] in ['블라우스', '셔츠']:
                label = '0'
            else:
                label = '1'
        # else:
        #     label = '1'
    elif i == '하의':
        if '카테고리' in labeling:
            if labeling['카테고리'] == '스커트':
                label = '6'
            elif labeling['카테고리'] in ['청바지','팬츠', '조거팬츠']:
                if '기장' in labeling:
                    if labeling['기장'] in ['미니', '니렝스']:
                        label = '5'
                    else:
                        label = '4'
                else: 
                    label = '4'
        # else:
        #     label = '4'
    elif i == '원피스':
        if '카테고리' in labeling:
            if labeling['카테고리'] == '드레스':
                label = '8'
        # else:
        #     label = '8'
    return label

In [4]:
path = r'C:\Users\user\Desktop\k-fashion\test'
print(os.listdir(path))
des_path = os.path.join(r'C:\Users\user\Desktop\k-fashion', 'labels_txt')
createDirectory(des_path)

['레트로', '로맨틱', '리조트']


In [5]:
for p in os.listdir(path):
    tmp_path = os.path.join(path, p)
    a = glob(os.path.join(tmp_path, '*.json'))

    for file in a:
        with open(file) as f:
            text = json.load(f)
        h, w = int(text['이미지 정보']['이미지 높이']), int(text['이미지 정보']['이미지 너비'])
        t_name = file[len(path)+len(p)+2:-5] + '.txt'
        m = text['데이터셋 정보']['데이터셋 상세설명']['폴리곤좌표']
        final_result = []
        for i in m:
            result=[]
            if m[i][0]:
                labeling = text['데이터셋 정보']['데이터셋 상세설명']['라벨링'][i][0]
                label = label_make(i, labeling)
                if label != '99':
                    result.append(label)
                    k = sorted(m[i][0].items())
                    k1 = k[:len(k)//2]
                    k2 = k[len(k)//2:]
                    tmp =[[0,0] for i in range(len(k)//2)]
                    for x_item, y_item in zip(k1, k2):
                        index = int(x_item[0][3:]) - 1
                        if index >= len(tmp):
                            y = [round(x_item[1]/w, 6), round(y_item[1]/h, 6)]
                            tmp.append(y)
                        else:
                            tmp[index][0] = round(x_item[1]/w, 6)
                            tmp[index][1] = round(y_item[1]/h, 6)
                    tmp = [j for j in tmp if j != [0,0]]
                    for point in tmp:
                        result.append(str(point[0]))
                        result.append(str(point[1]))
                    final_result.append(result)
        with open(os.path.join(des_path, t_name), 'w') as f:
            for line in final_result:
                f.write(' '.join(line)+'\n')

필요한거 : 상의 카테고리, 하의 카테고리, 아우터 카테고리, 원피스 카테고리

            컬러..??

            하의 기장, 아우터 기장
            
            소매기장..?

1. 일단 각 폴리곤 좌표가 비어있는지
2. 

### 상의 카테고리 분류
- 탑, 티셔츠, 니트웨어, 후드티, 브라탑
- 블라우스, 셔츠

* 상의 기장 신경 쓸 필요 없음
___

### 하의 카테고리
- 청바지, 팬츠, 레깅스, 조거팬츠 -> 레깅스 넣을 건지 판단
- 스커트

*  하의 기장 < 바지만! >
1. 미니 니렝스 
2. 미디 발목 맥시
___
### 아우터 카테고리 분류
- 코트, 패딩..?
- 재킷, 점퍼, 가디건, 짚업
- 베스트

* 현재 데이터셋의 아우터 분류는 기장으로 했으므로
이 데이터셋의 카테고리보다 기장으로 분류하는게 나을 수도 있음
___
### 원피스 카테고리 분류
- 드레스
- 점프수트는 배제

* 기장 필요 없음

___
### 컬러 필요하면 사용
